# Deep Learning: PyTorch Tutorials

PyTorch is an open-source machine learning framework developed by Meta AI, known for accelerating the path from research prototyping to production deployment. It’s beginner-friendly due to its Pythonic syntax and dynamic computation graph, which allows flexible model design and easy debugging. PyTorch supports GPU acceleration for faster computations and includes libraries like torchvision for computer vision tasks, making it ideal for deep learning applications such as image classification and natural language processing.

**Key Features**:  
- Dynamic Computation Graphs
- Tensors: Multi-dimensional arrays optimized for CPU and GPU operations
- Autograd: Automatic differentiation for computing gradients during training.
- Neural Network Modules: Tools like `nn.Module` simplify model construction.
- Ecosystem: Libraries for vision (`torchvision`), text (`torchtext`), and audio (`torchaudio`). 

In [1]:
# Verify the installation

import torch
print(torch.__version__)  # Displays PyTorch version
print(torch.cuda.is_available())  # Checks for GPU support

2.7.1+cpu
False


In [2]:
# Understanding Tensors

# Tensors are PyTorch’s core data structure, similar to NumPy arrays 
# but with GPU support and autograd capabilities. They represent multi-dimensional 
# arrays used for model inputs, outputs, and weights. Tensors support over 300 
# mathematical operations, optimized for performance in compiled C++ code.

# Creating and Manipulating Tensors

# You can create tensors from lists, arrays, or random values, and perform operations 
# like addition, multiplication, and reshaping. Tensors can be moved to GPUs for faster 
# computations, crucial for deep learning tasks.

# Create tensors
tensor_list = torch.tensor([1, 2, 3])  # From a list
zeros_tensor = torch.zeros(2, 3)       # 2x3 tensor of zeros
ones_tensor = torch.ones(2, 3)         # 2x3 tensor of ones
random_tensor = torch.rand(2, 3)       # 2x3 random tensor

# Basic operations
sum_tensor = tensor_list + 1           # Add scalar
product_tensor = tensor_list * 2       # Multiply by scalar
matmul_result = torch.matmul(ones_tensor, random_tensor.t())  # Matrix multiplication

# Reshape tensor
reshaped_tensor = tensor_list.view(3, 1)  # Reshape to 3x1

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tensor_gpu = tensor_list.to(device)

print("Tensor from list:", tensor_list)
print("Zeros tensor:", zeros_tensor)
print("Matrix multiplication:", matmul_result)
print("Reshaped tensor:", reshaped_tensor)

Tensor from list: tensor([1, 2, 3])
Zeros tensor: tensor([[0., 0., 0.],
        [0., 0., 0.]])
Matrix multiplication: tensor([[1.7769, 1.9634],
        [1.7769, 1.9634]])
Reshaped tensor: tensor([[1],
        [2],
        [3]])


Tensors are the data structure that makes deep learning tractable. Their ability to handle high-dimensional data, track gradients, and leverage hardware acceleration (via CUDA) is what allows complex models to process images, text, or time-series data efficiently.

In [3]:
# What is Autograd?

# Autograd is PyTorch’s automatic differentiation engine, which tracks operations 
# on tensors to compute gradients automatically. This simplifies backpropagation, 
# enabling neural networks to learn by adjusting weights based on the loss function’s 
# gradients.

# How Autograd Works

# When a tensor has requires_grad=True, PyTorch records its operations in a computation 
# graph. Calling .backward() computes gradients for all tensors in the graph, which are 
# stored in the .grad attribute. This is essential for optimizing model parameters 
# during training.

# Create a tensor with requires_grad=True
x = torch.tensor(2.0, requires_grad=True)

# Define a simple function
y = x ** 2 + 3 * x + 1

# Compute gradients
y.backward()

# Print gradient (dy/dx = 2x + 3, at x=2, gradient = 7)
print("Gradient of y with respect to x:", x.grad)

Gradient of y with respect to x: tensor(7.)


In [4]:
# Neural Networks with nn.Module

import torch.nn as nn
import torch.nn.functional as F

# Define a neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(4, 10)  # Input: 4 features, Output: 10 units
        self.fc2 = nn.Linear(10, 3)  # Output: 3 classes (iris dataset)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))  # ReLU activation
        x = self.fc2(x)          # Output logits
        return x

# Instantiate and test the model
model = SimpleNN()
sample_input = torch.rand(1, 4)  # Batch of 1, 4 features
output = model(sample_input)
print("Model output shape:", output.shape)  # Should be [1, 3]

Model output shape: torch.Size([1, 3])


In [5]:
# Code Example: Building, Training, and Evaluating on the Iris Dataset

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader

# Load and preprocess iris dataset
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create datasets and loaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Define model, loss, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy = correct / total
print(f'Test Accuracy: {accuracy:.2f}')

# Save the model
torch.save(model.state_dict(), 'iris_model.pth')

Epoch [10/100], Loss: 1.0595
Epoch [20/100], Loss: 0.9075
Epoch [30/100], Loss: 0.7088
Epoch [40/100], Loss: 0.5620
Epoch [50/100], Loss: 0.4821
Epoch [60/100], Loss: 0.4325
Epoch [70/100], Loss: 0.3846
Epoch [80/100], Loss: 0.3456
Epoch [90/100], Loss: 0.3078
Epoch [100/100], Loss: 0.2814
Test Accuracy: 0.97
